# Imports and loads

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('Documents\loons sub tracking.xlsx')

# Cleaning

In [3]:
data.columns = map(str.lower, data.columns)

In [4]:
data.columns = data.columns.str.replace(' ','_')

In [5]:
data = data.drop('unnamed:_15', axis=1)

In [6]:
data['sub_gamestate']='draw'

In [7]:
data['loon_score'] = data['score_at_sub'].str[0]
data['opp_score'] = data['score_at_sub'].str[2]

In [8]:
data[['loon_score','opp_score']] = data[['loon_score','opp_score']].apply(pd.to_numeric)

In [9]:
data['sub_gamestate'] = np.where(data['loon_score']>data['opp_score'],'Win',
                                 np.where(data['loon_score']<data['opp_score'],'Loss','Draw'))

# split dataset

In [10]:
    # break out player types, substitutions, and results

In [11]:
defenders = data.defenders
mid = data.midfieldsers
attack = data.forwards

In [12]:
subs = data.drop(['forwards','midfieldsers','defenders'], axis=1)

In [13]:
results = data[['date','opponent','final','result','sub_#']]

# Clean results dataframeto remove duplicates and count subs

In [14]:
results['max_sub'] = results.groupby(['date','opponent','final','result'])['sub_#'].transform(max)

c:\users\zak_laptop\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
results = results.drop('sub_#', axis=1)

In [16]:
results = results.drop_duplicates()

In [17]:
results['date'] = pd.to_datetime(results['date'])

In [67]:
subs['date']= pd.to_datetime(subs['date'])

In [68]:
subs['year'] = subs['date'].dt.year

# EDA

In [18]:
results['year'] = results['date'].dt.year

In [19]:
#Average # subs used per game
results.groupby(['year']).mean()

,max_sub
year,
2017,2.588235
2018,2.705882
2019,2.785714


In [20]:
#average # subs by result
results.groupby(['result']).mean()

,max_sub,year
result,,
Draw,2.500000,2017.750000
Loss,2.581395,2017.697674
Win,2.888889,2017.851852


In [21]:
#average # subs by year and result
results.groupby(['year','result']).mean()

max_sub
year result          
2017 Draw    2.500000
     Loss    2.444444
     Win     2.900000
2018 Draw    2.333333
     Loss    2.700000
     Win     2.818182
2019 Draw    2.666667
     Loss    2.600000
     Win     3.000000

In [22]:
subs.head()

,date,opponent,final,result,time,sub_#,sub_on,sub_off,movement,score_at_sub,score_change,on_index,off_index,end_diff,sub_diff,sub_gamestate,loon_score,opp_score
0,2019-05-29,Atlanta,0-3,Loss,64,1,Toye,Danladi,0,0-1,-2,2,2,-3,-1,Loss,0,1
1,2019-05-29,Atlanta,0-3,Loss,70,2,Rodriguez,Quintero,0,0-1,-2,2,2,-3,-1,Loss,0,1
2,2019-05-29,Atlanta,0-3,Loss,82,3,Finlay,Schuller,1,0-1,-2,2,1,-3,-1,Loss,0,1
3,2018-07-15,RSL,3-2,Win,72,1,Martin,Ibson,0,3-0,-2,1,1,1,3,Win,3,0
4,2018-07-15,RSL,3-2,Win,78,2,Fernando,Schuller,0,3-1,-1,1,1,1,2,Win,3,1


In [79]:
#subs made before halftime are removed as they are not necessarily made tactically, but still count agaisnt the 3 allowed
subs_no_injury = subs.drop(subs[(subs.time < 45)].index)

In [24]:
#average time of subs
subs.groupby(['sub_#'])['time'].mean()

sub_#
1    59.207317
2    73.227848
3    82.017241
Name: time, dtype: float64

In [25]:
#average time of subs excluding first half injuries
subs_no_injury.groupby(['sub_#'])['time'].mean()

sub_#
1    64.911765
2    73.602564
3    82.017241
Name: time, dtype: float64

In [83]:
#Average time of sub by result
subs.groupby(['sub_gamestate','sub_#'])['time','movement'].mean()

time  movement
sub_gamestate sub_#                     
Draw          1      50.000000 -0.411765
              2      73.785714 -0.214286
              3      83.300000 -0.400000
Loss          1      58.023810  0.309524
              2      69.780488  0.195122
              3      78.740741  0.481481
Win           1      68.173913 -0.434783
              2      78.791667 -0.166667
              3      85.619048 -0.619048

In [84]:
#Average time of sub by result excluding first half injuries
subs_no_injury.groupby(['sub_gamestate','sub_#'])['time','movement'].mean()

time  movement
sub_gamestate sub_#                     
Draw          1      65.800000 -0.600000
              2      73.785714 -0.214286
              3      83.300000 -0.400000
Loss          1      61.351351  0.378378
              2      70.425000  0.200000
              3      78.740741  0.481481
Win           1      70.761905 -0.428571
              2      78.791667 -0.166667
              3      85.619048 -0.619048

# changes in games due to subs

In [28]:
#end state and sub state must be different
gamechanges= subs.drop(subs[(subs.result==subs.sub_gamestate)].index)

In [29]:
#positive subs don't end in a loss
possubs = gamechanges.drop(gamechanges[(gamechanges.result == 'Loss')].index)

In [30]:
#positive can't have come on when we're winning
possubs = possubs.drop(possubs[(possubs.sub_gamestate == 'Win')].index)

In [31]:
#number of total subs leading to gained points
len(possubs.index)

24

In [32]:
#inverse process for negative changes

In [33]:
negsubs = gamechanges.drop(gamechanges[(gamechanges.result == 'Win')].index)

In [34]:
negsubs = negsubs.drop(negsubs[(negsubs.sub_gamestate =='Loss')].index)

In [35]:
#total number of subs leading to lost points
len(negsubs.index)

15

In [36]:
#find points earned from subs made, max points taken per game

In [37]:
possubs['end_points'] = np.where(possubs['result'] == 'Win',3,1)
possubs['sub_points'] = np.where(possubs['sub_gamestate'] == 'Draw',1,0)

In [38]:
possubs['points_gained'] = possubs['end_points'] - possubs['sub_points']

In [39]:
points_from_subs = possubs[['date','points_gained']]

In [87]:
len(points_from_subs.index)

24

In [40]:
points_from_subs.sort_values('points_gained', ascending=False).drop_duplicates('date').sort_index()

,date,points_gained
17,2019-04-28,2
57,2018-09-16,1
67,2018-08-12,1
74,2018-07-23,2
108,2018-04-29,3
127,2018-03-11,2
141,2017-10-08,1
144,2017-10-04,3
151,2017-09-17,2
156,2017-08-27,2


In [41]:
points_from_subs.points_gained.sum()

41

In [42]:
#find total points lost on bad subs

In [43]:
negsubs['end_points'] = np.where(negsubs['result'] == 'Draw',1,0)
negsubs['sub_points'] = np.where(negsubs['sub_gamestate'] == 'Win',3,1)

In [44]:
negsubs['points_lost'] = negsubs['end_points'] - negsubs['sub_points']

In [45]:
lost_points_from_subs = negsubs[['date','points_lost']]

In [46]:
lost_points_from_subs.sort_values('points_lost', ascending=False).drop_duplicates('date').sort_index()

,date,points_lost
25,2019-04-13,-2
58,2018-09-13,-1
61,2018-08-26,-1
69,2018-08-05,-3
90,2018-06-24,-1
159,2017-08-21,-1
172,2017-07-04,-1
180,2017-06-18,-1
181,2017-06-03,-1
188,2017-05-21,-1


In [47]:
lost_points_from_subs.points_lost.sum()

-22

In [48]:
# most subs on and off

In [49]:
subs.sub_on.value_counts()

Danladi         29
Toye            16
Warner          16
Ibarra          14
Romario         11
Martin          11
Venegas          9
Schuller         9
Taylor           8
Pangop           8
Nicholson        6
Ibson            6
Kadrii           6
Ramirez          6
Finlay           6
Miller           6
Rodriguez        5
Jome             5
Fernando         5
Manley           4
Kallman          4
Dotson           4
Davis            3
Omsberg          3
Heath            3
Burch            2
Quintero         2
Gomez            2
Kevin            1
Alvbage          1
Shuttleworth     1
Leiton           1
Molino           1
Opara            1
Thiesson         1
Greenspan        1
Mears            1
Allen            1
Name: sub_on, dtype: int64

In [50]:
subs.sub_off.value_counts()

Ramirez         21
Ibarra          20
Schuller        20
Danladi         17
Molino          15
Quintero        13
Nicholson       10
Ibson           10
Finlay           9
Rodriguez        9
Martin           8
Gomez            8
Kallman          6
Miller           6
Burch            5
Warner           5
Venegas          5
Jome             4
Kadrii           4
Romario          4
Fernando         4
Thiesson         3
Cronin           2
Mears            2
Shuttleworth     1
Taylor           1
Omsberg          1
Greenspan        1
Pangop           1
Opara            1
Toye             1
Saeid            1
Alvbage          1
Name: sub_off, dtype: int64

In [53]:
subs['sub_combo'] = subs['sub_on'] + subs['sub_off']

In [52]:
subs.sub_combo.value_counts()

ToyeRamirez         8
DanladiRamirez      6
WarnerIbson         4
FinlaySchuller      4
RamirezDanladi      4
WarnerMartin        4
DanladiRodriguez    4
RomarioMiller       3
DanladiQuintero     3
IbarraNicholson     3
RodriguezRomario    3
DanladiMolino       3
WarnerDanladi       3
IbarraMolino        3
KadriiJome          3
DanladiIbarra       3
DanladiVenegas      2
IbsonSchuller       2
RomarioIbarra       2
MartinIbson         2
DotsonQuintero      2
VenegasIbarra       2
FernandoGomez       2
MartinSchuller      2
NicholsonIbarra     2
NicholsonBurch      2
TaylorKallman       2
VenegasRamirez      2
SchullerDanladi     2
JomeIbarra          2
                   ..
TaylorRamirez       1
MillerRamirez       1
RamirezSchuller     1
ToyeOmsberg         1
DavisBurch          1
NicholsonMartin     1
VenegasNicholson    1
JomeGreenspan       1
GreenspanKallman    1
ToyeSchuller        1
IbsonFernando       1
ToyeNicholson       1
MillerMolino        1
MillerQuintero      1
DanladiToy

In [70]:
subs.groupby(['year','sub_gamestate'])['movement'].mean()

year  sub_gamestate
2017  Draw            -0.350000
      Loss             0.173913
      Win             -0.545455
2018  Draw            -0.461538
      Loss             0.450980
      Win             -0.250000
2019  Draw            -0.125000
      Loss             0.230769
      Win             -0.444444
Name: movement, dtype: float64

In [74]:
subs.groupby(['year','result'])['movement'].mean()

year  result
2017  Draw      0.066667
      Loss     -0.068182
      Win      -0.310345
2018  Draw      0.142857
      Loss      0.351852
      Win      -0.322581
2019  Draw     -0.125000
      Loss      0.230769
      Win      -0.444444
Name: movement, dtype: float64

In [85]:
subs['sub_type'] = np.where(subs['movement']>0,'attacking',np.where(subs['movement']<0,'defending','neutral'))